In [1]:
import findspark
findspark.init('/home/mint/spark-2.1.0-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [4]:
lr = LinearRegression(featuresCol = 'features', labelCol = 'label', predictionCol = 'prediction')
lrModel = lr.fit(training)

In [5]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [6]:
lrModel.intercept

0.14228558260358093

In [8]:
training_summary = lrModel.summary
training_summary.rootMeanSquaredError

10.16309157133015

In [10]:
## Train test split
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [11]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                345|
|   mean| 0.9114371787451888|
| stddev| 10.269434930373443|
|    min|-28.571478869743427|
|    max| 27.111027963108548|
+-------+-------------------+



In [12]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                156|
|   mean|-1.1906700432525703|
| stddev| 10.310027763467124|
|    min|-26.805483428483072|
|    max|  27.78383192005107|
+-------+-------------------+



In [15]:
correct_model = lr.fit(train_data)
test_results = correct_model.evaluate(test_data)
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-28.734430170540694|
|-23.122561826756126|
| -24.33860394786413|
|-20.058456052824997|
| -22.05082023438219|
|-21.672612179869585|
|-20.778102272224928|
|-21.141325485555086|
| -20.79303734392628|
| -18.32770149645428|
| -16.56595246163561|
|-17.038428881806507|
|-15.611749832205128|
|-17.982754192758826|
|-18.066917295458133|
|-15.439944374421009|
|-16.963672665945538|
| -15.52374479214914|
|-14.732473409712057|
|-14.321189008119982|
+-------------------+
only showing top 20 rows



In [14]:
test_results.rootMeanSquaredError

10.411158024523184

In [16]:
unlabeled_data = test_data.select('features')
prediction = correct_model.transform(unlabeled_data)
prediction.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  1.9289467420576214|
|(10,[0,1,2,3,4,5,...|-0.38832226356684385|
|(10,[0,1,2,3,4,5,...|  0.8511638269276162|
|(10,[0,1,2,3,4,5,...|  -2.779004364094345|
|(10,[0,1,2,3,4,5,...|   1.993337618592978|
|(10,[0,1,2,3,4,5,...|  1.7996211418011794|
|(10,[0,1,2,3,4,5,...|  0.9953394826103897|
|(10,[0,1,2,3,4,5,...|  1.4740068701833648|
|(10,[0,1,2,3,4,5,...|  1.9471148710276989|
|(10,[0,1,2,3,4,5,...|  1.2623018705782645|
|(10,[0,1,2,3,4,5,...|-0.15314437196947817|
|(10,[0,1,2,3,4,5,...|  0.9527698407850151|
|(10,[0,1,2,3,4,5,...| -0.3397627335894463|
|(10,[0,1,2,3,4,5,...|   2.120744865188265|
|(10,[0,1,2,3,4,5,...|  2.3434016824095623|
|(10,[0,1,2,3,4,5,...|0.002559580989791...|
|(10,[0,1,2,3,4,5,...|  1.9071896914031026|
|(10,[0,1,2,3,4,5,...|  0.7015918823979502|
|(10,[0,1,2,3,4,5,...|  0.7563424785593541|
|(10,[0,1,2,3,4,5,...|  0.900594